# GarmentIQ Landmark Detection Quick Start

In [ ]:
import garmentiq as giq
from garmentiq.landmark.detection.model_definition import PoseHighResolutionNet
from garmentiq.garment_classes import garment_classes

In [ ]:
# Download a vest dress image and a pretrained model
!mkdir -p test_image
!wget -q -O /app/working/examples/landmark_refinement_and_derivation_quick_start/test_image/cloth_3.jpg \
    https://raw.githubusercontent.com/lygitdata/GarmentIQ/refs/heads/gh-pages/asset/img/cloth_3.jpg

!mkdir -p models
!wget -q -O /app/working/examples/landmark_refinement_and_derivation_quick_start/models/hrnet.pth \
    https://huggingface.co/lygitdata/garmentiq/resolve/main/hrnet.pth

In [ ]:
# Plot the image
giq.landmark.plot(
    image_path="/app/working/examples/landmark_refinement_and_derivation_quick_start/test_image/cloth_3.jpg", 
    figsize=(3, 3)
)

In [ ]:
# Load the pretrained model from Hugging Face
HRNet = giq.landmark.detection.load_model(
    model_path="/app/working/examples/landmark_refinement_and_derivation_quick_start/models/hrnet.pth",
    model_class=PoseHighResolutionNet()
)

# Detect predefined landmarks
coords, maxvals, detection_dict = giq.landmark.detect(
    class_name="vest dress",
    class_dict=garment_classes,
    image_path="/app/working/examples/landmark_refinement_and_derivation_quick_start/test_image/cloth_3.jpg",
    model=HRNet,
    scale_std=200.0,
    resize_dim=[288, 384],
    normalize_mean=[0.485, 0.456, 0.406],
    normalize_std=[0.229, 0.224, 0.225]
)

# Plot the detected coordinates
giq.landmark.plot(
    image_path="/app/working/examples/landmark_refinement_and_derivation_quick_start/test_image/cloth_3.jpg", 
    coordinate=coords, 
    figsize=(3, 3), 
    color="green"
)

In [ ]:
# Segmentation mask is required for refinement and derivation
# So we need to do segmentation first
BiRefNet = giq.segmentation.load_model(
    pretrained_model='lygitdata/BiRefNet_garmentiq_backup',
    pretrained_model_args={'trust_remote_code': True},
    high_precision=True
)
original_img, mask = giq.segmentation.extract(
    model=BiRefNet,
    image_path='/app/working/examples/landmark_refinement_and_derivation_quick_start/test_image/cloth_3.jpg',
    resize_dim=(1024, 1024),
    normalize_mean=[0.485, 0.456, 0.406],
    normalize_std=[0.229, 0.224, 0.225],
    high_precision=True
)

In [ ]:
# Refine the landmarks
refined_coords, refined_detection_dict = giq.landmark.refine(
    class_name="vest dress",
    detection_np=coords,
    detection_conf=maxvals,
    detection_dict=detection_dict,
    mask=mask,
    window_size=5,
    ksize=(11, 11),
    sigmaX=0.0
)

# Print the original coordinates and the refined coordinates
print("Original coordinates:\n", coords)
print("Refined coordinates:\n", refined_coords)

In [ ]:
# Derive custom landmarks
derived_coords, derived_detection_dict = giq.landmark.derive(
    class_name="vest dress",
    detection_dict=refined_detection_dict,
    derivation_dict=giq.landmark.derivation_dict.derivation_dict,
    landmark_coords=refined_coords,
    np_mask=mask
)

In [ ]:
import numpy as np

# Plot the derived point
giq.landmark.plot(
    image_path="/app/working/examples/landmark_refinement_and_derivation_quick_start/test_image/cloth_3.jpg",
    coordinate=np.concatenate((refined_coords, np.array([[derived_coords['20']]])), axis=1),
    figsize=(3, 3),
    color="green"
)